
# Modelado de Datos: Capa Gold en Azure Databricks

Este notebook toma los datos ya transformados de la **capa Silver** y los organiza en un modelo de datos consumible para análisis, usualmente siguiendo un enfoque de modelo estrella o constelación (star schema), y escribe los resultados en la **capa Gold** utilizando Delta Lake.


In [0]:
from pyspark.sql.functions import datediff
from delta.tables import DeltaTable

In [0]:
from typing import Optional
from pyspark.sql.functions import col, current_timestamp, expr, lit
from functools import reduce
from operator import and_

def leer_desde_silver(
    nombre_tabla: str,
    catalog_name: str = "desarrollo",
    db_silver: str = "silver_clinica",
    mode: str = "full",                 # "full" | "diff"
    ts_col: str = "fecalta",
    last_n_days: Optional[int] = None,  # ej. 2  -> últimos 2 días
    last_n_hours: Optional[int] = None, # ej. 6  -> últimas 6 horas
    since: Optional[str] = None,        # "YYYY-MM-DD" o "YYYY-MM-DD HH:mm:ss"
    until: Optional[str] = None,        # límite superior EXCLUSIVO
    drop_nulls: bool = True
):
    full = f"{catalog_name}.{db_silver}.{nombre_tabla}"
    if not spark.catalog.tableExists(full):
        raise ValueError(f"La tabla {full} no existe en el metastore.")

    df = spark.table(full)

    # Validar tipo timestamp
    dtype = dict(df.dtypes).get(ts_col, "").lower()
    if dtype != "timestamp":
        raise ValueError(f"La columna '{ts_col}' debe ser timestamp. Actual: {dtype}")

    if mode.lower() == "full":
        return df if not drop_nulls else df.filter(col(ts_col).isNotNull())

    # --- Diferencial por timestamp ---
    conds = []
    if drop_nulls:
        conds.append(col(ts_col).isNotNull())

    if last_n_days is not None:
        conds.append(col(ts_col) >= (current_timestamp() - expr(f"INTERVAL {int(last_n_days)} DAYS")))
    if last_n_hours is not None:
        conds.append(col(ts_col) >= (current_timestamp() - expr(f"INTERVAL {int(last_n_hours)} HOURS")))
    if since:
        conds.append(col(ts_col) >= lit(since).cast("timestamp"))
    if until:
        conds.append(col(ts_col) <  lit(until).cast("timestamp"))  # upper bound abierto

    if not conds:
        raise ValueError("En mode='diff' especifica last_n_days/last_n_hours o since/until.")

    return df.filter(reduce(and_, conds))


In [0]:
# Full
#df_full = leer_desde_silver("t_camas", db_silver="clinica_silver", mode="full")

# Diferencial: últimos 2 días por 'fecalta'
#df_diff_2d = leer_desde_silver("t_camas", "clinica_silver", mode="diff", date_col="fecalta", last_n_days=2)

# Diferencial: rango específico [2025-08-20, 2025-08-23)
#df_diff_range = leer_desde_silver("t_camas", "clinica_silver", mode="diff",
 #                                 date_col="fecalta", start="2025-08-20", end="2025-08-23")

# Si 'fecalta' es string tipo "YYYY-MM-DD HH:mm:ss", ajusta el formato si difiere:
# string_datetime_fmt="dd/MM/yyyy HH:mm:ss"


In [0]:
from pyspark.sql.functions import col, to_timestamp, when

# Leer desde silver
#df_Camas = leer_desde_silver("md_camas", "clinica_silver", mode="diff",
#                           since="2025-08-20 00:00:00", until="2025-08-23 00:00:00")
df_Camas = leer_desde_silver("md_camas", db_silver="silver_clinica", mode="full")

# Mostrar la estructura del DataFrame después de la limpieza
df_Camas.printSchema()

# Contar filas 
print(f"Total de filas: {df_Camas.count()}")

In [0]:
# Leer desde bronze
df_Habitaciones = leer_desde_silver("md_habitaciones", db_silver="silver_clinica", mode="full")

# Mostrar la estructura del DataFrame después de la limpieza
df_Habitaciones.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_Habitaciones.count()}")

In [0]:
# Leer desde silver
df_Medicos = leer_desde_silver("md_medicos", db_silver="silver_clinica", mode="full")

# Mostrar la estructura del DataFrame después de la limpieza
df_Medicos.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_Medicos.count()}")

In [0]:
# Leer desde silver
df_Internamientos = leer_desde_silver("md_internamientos", db_silver="silver_clinica", mode="full")

# Mostrar la estructura del DataFrame después de la limpieza
df_Internamientos.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_Internamientos.count()}")

In [0]:
# Leer desde silver
df_Pacientes = leer_desde_silver("md_pacientes", db_silver="silver_clinica", mode="full")

# Mostrar la estructura del DataFrame después de la limpieza
df_Pacientes.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_Pacientes.count()}")

In [0]:
# Leer desde silver
df_VisitasMedicas = leer_desde_silver("md_visitas_medicas", db_silver="silver_clinica", mode="full")

# Mostrar la estructura del DataFrame después de la limpieza
df_VisitasMedicas.printSchema()

# Contar filas antes y después de la limpieza para validar cambios
print(f"Total de filas después de la limpieza: {df_VisitasMedicas.count()}")

In [0]:
# Realizamos left join: habitaciones ← camas
df_HabitacionCama = df_Habitaciones.join(
    df_Camas,
    df_Habitaciones["numero_habitacion"] == df_Camas["numero_habitacion"],
    how="left"
)

# (Opcional) eliminamos columna duplicada si existe
df_HabitacionCama = df_HabitacionCama.drop(df_Camas["numero_habitacion"],df_Camas["year"], df_Camas["month"], df_Camas["day"], df_Camas["fecalta"], df_Camas["usralta"], df_Camas["estado"])

# Mostrar resultado
df_HabitacionCama.show(truncate=False)


In [0]:
from pyspark.sql.functions import col, min as min_, sequence, explode, to_date, year, month, dayofmonth, quarter, lit
from pyspark.sql.types import DateType
from datetime import datetime

def crear_dim_tiempo(df_par):
    """
    Crea un DataFrame de dimensión tiempo desde la primera fecha de 'fecalta' hasta hoy.

    Parámetros:
    - df_internamientos: DataFrame con columna 'fecalta'

    Retorna:
    - DataFrame con columnas: fecha, año, mes, día, trimestre
    """
    # Obtener fecha mínima de alta desde el DataFrame
    fecha_min = df_par.select(min_("fecalta")).first()[0]
    fecha_min_str = fecha_min.strftime("%Y-%m-%d") if isinstance(fecha_min, datetime) else fecha_min

    # Crear DataFrame con rango de fechas desde fecha mínima hasta hoy
    df_rango = spark.sql(f"SELECT sequence(to_date('{fecha_min_str}'), current_date(), interval 1 day) AS fechas")
    df_fechas = df_rango.select(explode(col("fechas")).alias("fecha"))

    # Agregar columnas de atributos de tiempo
    df_dim_tiempo = df_fechas.select(
        col("fecha").alias("fecha"),
        year("fecha").alias("anio"),
        month("fecha").alias("mes"),
        dayofmonth("fecha").alias("dia"),
        quarter("fecha").alias("trimestre")
    )

    return df_dim_tiempo


In [0]:
df_dim_tiempo = crear_dim_tiempo(df_Internamientos)
df_dim_tiempo.show()


In [0]:
%sql
USE CATALOG desarrollo;

SHOW DATABASES;

In [0]:
from typing import List, Optional
from delta.tables import DeltaTable

def crear_tabla_delta_merge_managed(
    nombre_df: str,
    nombre_tabla: str,
    llave_origen: List[str],
    llave_destino: List[str],
    db_name: str = "default",
    catalog_name: str = "desarrollo",
    partition_cols: Optional[List[str]] = None,
    auto_merge_schema: bool = True
) -> None:
    """
    Crea si no existe una tabla Delta GESTIONADA en la base (que ya debe tener LOCATION en tu mount)
    y realiza MERGE. No usa LOCATION explícito.
    """

    # Validaciones
    df = globals()[nombre_df]

    if len(llave_origen) != len(llave_destino):
        print("❌ Error: La cantidad de columnas en 'llave_origen' y 'llave_destino' no coinciden.")
        return

    if partition_cols:
        faltantes = [c for c in partition_cols if c not in df.columns]
        if faltantes:
            print(f"❌ Error: Columnas de partición no existen en el DataFrame: {faltantes}")
            return

    if auto_merge_schema:
        spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    # Armar nombre completo
    full_name = f"{catalog_name}.{db_name}.{nombre_tabla}"

    # ✅ FIX: usar el overload moderno (una sola cadena)
    exists = spark.catalog.tableExists(full_name)

    if not exists:
        # Crear como TABLA GESTIONADA en el LOCATION de la DB (sin LOCATION explícito)
        writer = df.write.format("delta").mode("overwrite")
        if partition_cols:
            # ✅ FIX: varargs
            writer = writer.partitionBy(*partition_cols)
        writer.saveAsTable(full_name)
        print(f"✅ Tabla gestionada creada: {full_name} (bajo LOCATION de la base '{db_name}')")
        return

    # Si existe, MERGE
    try:
        delta_tbl = DeltaTable.forName(spark, full_name)
    except Exception as e:
        raise RuntimeError(f"❌ La tabla {full_name} no es Delta o no es accesible como Delta: {e}")

    merge_condition = " AND ".join(
        [f"tgt.`{llave_destino[i]}` = src.`{llave_origen[i]}`" for i in range(len(llave_origen))]
    )
    set_expr  = {c: f"src.`{c}`" for c in df.columns}
    vals_expr = {c: f"src.`{c}`" for c in df.columns}

    print(f"🔄 Ejecutando MERGE INTO {full_name} ...")
    (delta_tbl.alias("tgt")
             .merge(df.alias("src"), merge_condition)
             .whenMatchedUpdate(set=set_expr)
             .whenNotMatchedInsert(values=vals_expr)
             .execute())
    print(f"✅ MERGE completado para {full_name}")


In [0]:
# Ejecutar la función para crear la tabla y hacer MERGE usando diferentes llaves
crear_tabla_delta_merge_managed(
    nombre_df="df_HabitacionCama",
    nombre_tabla="dim_habitacion",
    llave_origen=["numero_cama"],
    llave_destino=["numero_cama"],
    db_name="gold_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Pacientes",
    nombre_tabla="dim_pacientes",
    llave_origen=["id_paciente"],
    llave_destino=["id_paciente"],
    db_name="gold_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Medicos",
    nombre_tabla="dim_medicos",
    llave_origen=["id_medico"],
    llave_destino=["id_medico"],
    db_name="gold_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_Internamientos",
    nombre_tabla="fact_internamientos",
    llave_origen=["id_internamiento"],
    llave_destino=["id_internamiento"],
    db_name="gold_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_VisitasMedicas",
    nombre_tabla="fact_visitas_medicas",
    llave_origen=["id_visita"],
    llave_destino=["id_visita"],
    db_name="gold_clinica",
    partition_cols=["fecalta"]  # opcional; si no quieres partición, quítalo
)

crear_tabla_delta_merge_managed(
    nombre_df="df_dim_tiempo",
    nombre_tabla="dim_tiempo",
    llave_origen=["fecha"],
    llave_destino=["fecha"],
    db_name="gold_clinica",
    partition_cols=["fecha"]  # opcional; si no quieres partición, quítalo
)